## Exploring Data with DataFrames and Spark SQL

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("GLIM_PGPBABI").enableHiveSupport().getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1484897226601_0012,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


### Read data from a Hive Table

In [2]:
student_df = spark.sql("SELECT * FROM studentmaster")

### Use DataFrame Methods
Spark DataFrames provide functions that you can use to extract and manipulate data. For example, you can use the **select** function to return a new DataFrame containing columns selected from an existing DataFrame.

In [3]:
student_df.select("School", "Reason").show()

+------+------+
|School|Reason|
+------+------+
|     0|     0|
|     0|     0|
|     0|     1|
|     0|     1|
|     0|     1|
|     0|     3|
|     0|     3|
|     0|     1|
|     0|     3|
|     0|     3|
|     0|     1|
|     0|     2|
|     0|     2|
|     0|     0|
|     0|     2|
|     0|     3|
|     0|     0|
|     0|     0|
|     0|     1|
|     0|     0|
+------+------+
only showing top 20 rows

In [5]:
student_df.filter(student_df['G3'] >=15).count()

376

### Combine Operations
In this case, you will use the **filter** function followed by the **select** and **describe** functions to return the summary statistics for the three exam scores for one of the schools

In [9]:
summary_stats_by_school = student_df.filter(student_df['School'] == 1).select('G1','G2','G3').describe()

In [10]:
summary_stats_by_school.show()

+-------+------------------+------------------+------------------+
|summary|                G1|                G2|                G3|
+-------+------------------+------------------+------------------+
|  count|              1160|              1160|              1160|
|   mean|10.000862068965517|10.229310344827587|10.458620689655172|
| stddev|2.7076409214911457| 2.938102233645812| 3.450377152748175|
|    min|                 4|                 0|                 0|
|    max|                19|                18|                19|
+-------+------------------+------------------+------------------+

### Count the Rows in a DataFrame
Now that you're familiar with working with DataFrames, a key task when building predictive solutions is to explore the data, determing statistics that will help you understand the data before building predictive models. For example, how many rows of flight data do you actually have?

In [11]:
student_df.count()

2320

### Determine Summary Statistics
Predictive modeling is based on statistics and probability, so you will often start by looking at summary statistics. The **describe** function returns a DataFrame containing the **count**, **mean**, **standard deviation**, **minimum**, and **maximum** values for each numeric column.

In [19]:
student_df.select('G1','G2','G3').describe().show()

+-------+------------------+------------------+------------------+
|summary|                G1|                G2|                G3|
+-------+------------------+------------------+------------------+
|  count|              2320|              2320|              2320|
|   mean|10.795689655172414|10.950431034482758|11.306034482758621|
| stddev| 2.691333486659969| 2.813852535705715|3.1378227577428275|
|    min|                 0|                 0|                 0|
|    max|                19|                19|                19|
+-------+------------------+------------------+------------------+

### Determine the Presence of Duplicates
The data you have to work with won't always be perfect - often you'll want to *clean* the data; for example to detect and remove duplicates that might affect your model. You can use the **dropDuplicates** function to create a new DataFrame with the duplicates removed, enabling you to determine how many rows are duplicates of other rows.

In [14]:
student_df.count() - student_df.dropDuplicates().count()

0

### Identify Missing Values
As well as determing if duplicates exist in your data, you should detect missing values, and either remove rows containing missing data or replace the missing values with a suitable relacement. The **dropna** function creates a DataFrame with any rows containing missing data removed - you can specify a subset of columns, and whether the row should be removed in *any* or *all* values are missing. You can then use this new DataFrame to determine how many rows contain missing values.

In [15]:
student_df.count() - student_df.dropDuplicates().dropna(how="any", subset=['G1','G2','G3']).count()

0

### Clean the Data
If there are duplicates and missing values, you can clean the data by removing the duplicates and replacing the missing values. The **fillna** function replaces missing values with a specified replacement value. In this case, you'll remove all duplicate rows and replace missing **G1**, **G2** and **G3** values with **0**.

### Explore Relationships in the Data
Predictive modeling is largely based on statistical relationships between fields in the data. To design a good model, you need to understand how the data points relate to one another and identify any apparent correlation. The **corr** function calculates a correlation value between -1 and 1, indicating the strength of correlation between two fields. A strong positive correlation (near 1) indicates that high values for one column are often found with high values for the other, which a string negative correlation (near -1) indicates that *low* values for one column are often found with *high* values for the other. A correlation near 0 indicates little apparent relationship between the fields.

In [21]:
student_df.corr("g2", "g3")

0.9244887496213747

### Use Spark SQL
In addition to using the DataFrame API directly to query data, you can persist DataFrames as table and use Spark SQL to query them using the SQL language. SQL is often more intuitive to use when querying tabular data structures.

In [22]:
student_df.createOrReplaceTempView("student_view")

In [23]:
spark.sql("SELECT School, COUNT(g3) AS NumStudents, AVG(g3) AS AvgScore FROM student_view GROUP BY School ORDER BY School").show()

+------+-----------+------------------+
|School|NumStudents|          AvgScore|
+------+-----------+------------------+
|     0|       1160|12.153448275862068|
|     1|       1160|10.458620689655172|
+------+-----------+------------------+

### Use the Inline SQL *Magic*
Jupyter Notebooks support *magics*, which enable you to include inline code and functionality. For example, the **%%sql** magic enables you to write SQL queries and visualize the results directly in the notebook.

Run the following query, and view the table of results that is returned.

In [26]:
%%sql
SELECT g2, g3 FROM student_view

Change the **Table** visualization of results above to a **Scatter** visualization to see the relationship between the **G2** and **G3** values more clearly (use the **-** function to plot the actual values) - visualizations like this make it easier to show relationships as apparent *structure* in the data. For example, the positive correlation between **G2** and **G3** seems to be a linear relationsip, creaing a diagonal line of plotted points.

### Query Multiple Tables
You can create and query multiple temporary tables. Run the cells below to create a temporary table from the **airports** DataFrame, and then use an inline query to query it together with the flights data.

In [27]:
school_df = spark.sql("SELECT * FROM school")
reason_df = spark.sql("SELECT * FROM reason")

In [34]:
school_df.createOrReplaceTempView("school_view")
reason_df.createOrReplaceTempView("reason_view")

In [39]:
%%sql
SELECT b.Category AS JoiningReason, AVG(a.g3) AS AvgScore
FROM student_view AS a JOIN reason_view AS b
ON a.Reason = b.LabelID
GROUP BY b.Category
ORDER BY AVG(a.G3) DESC

As you saw previously, it can sometimes be useful to visualize the results of a query. Change the visualization above to a **Bar** chart, using the **-** function, to see the average scores achieved by students based on the reasons for joining.

In [41]:
%%sql
SELECT b.Category AS SchoolName, c.Category AS JoiningReason, AVG(a.G3) AS AvgScore
FROM student_view AS a JOIN school_view AS b
ON a.School = b.LabelID JOIN reason_view AS c
ON a.Reason = c.LabelID
GROUP BY b.Category, c.Category
ORDER BY b.Category, AVG(a.G3) DESC